In [1]:
import pandas as pd

## Lire les donnees via l'api

In [6]:
from milestone1.question1 import get_play_by_play_season_gameType

match_type = 2

get_play_by_play_season_gameType("2015","0"+str(match_type),"data_saved/play_by_play")
get_play_by_play_season_gameType("2016","0"+str(match_type),"data_saved/play_by_play")
get_play_by_play_season_gameType("2017","0"+str(match_type),"data_saved/play_by_play")
get_play_by_play_season_gameType("2018","0"+str(match_type),"data_saved/play_by_play")
get_play_by_play_season_gameType("2019","0"+str(match_type),"data_saved/play_by_play")

 13%|█▎        | 160/1230 [00:40<04:31,  3.95it/s]


KeyboardInterrupt: 

## Creer les dataframes de test et train/val

In [2]:
from milestone1.question4 import *

#les donnes des saisons régulière seulement!
directory2015 = r'data_saved/play_by_play/2015/regular'
directory2016 = r'data_saved/play_by_play/2016/regular'
directory2017 = r'data_saved/play_by_play/2017/regular'
directory2018 = r'data_saved/play_by_play/2018/regular'
directory2019 = r'data_saved/play_by_play/2019/regular'


df2015 = create_full_df(directory=directory2015)
df2016 = create_full_df(directory=directory2016)
df2017 = create_full_df(directory=directory2017)
df2018 = create_full_df(directory=directory2018)

# 2019/20 comme l' ensemble final des données test.
df2019 = create_full_df(directory=directory2019)


In [7]:
#Question 4.4 Bonus
from question4 import *

df2015 = add_power_play(df2015, directory2015)
df2016 = add_power_play(df2016, directory2016)
df2017 = add_power_play(df2017, directory2017)
df2018 = add_power_play(df2018, directory2018)
df2019 = add_power_play(df2019, directory2019)

KeyboardInterrupt: 

In [42]:
df15_16 = pd.concat([df2015, df2016], ignore_index=True)
df17_18 = pd.concat([df2017, df2018], ignore_index=True)

#utilise les données 2015/16 - 2018/19 de la saison régulière pour créer vos ensembles d'entraînement et de validation.
df_train_val =  pd.concat([df15_16, df17_18], ignore_index=True)

In [24]:
print(df_train_val.head())

      Game_ID  Event_ID  Period_Number Period_Time  Game_Time Shot_or_Goal  \
0  2015020001         7              1       00:51       0.85         Shot   
1  2015020001         9              1       01:05       1.08         Shot   
2  2015020001        10              1       01:06       1.10         Shot   
3  2015020001        13              1       01:53       1.88         Shot   
4  2015020001        21              1       02:37       2.61         Shot   

    Shot_Type        Shooter Team_of_Shooter       Goalie  ... Goal_Strength  \
0  Wrist Shot     Brad Boyes             TOR  Carey Price  ...           NaN   
1   Snap Shot    Nazem Kadri             TOR  Carey Price  ...           NaN   
2   Snap Shot    Nazem Kadri             TOR  Carey Price  ...           NaN   
3  Wrist Shot  Daniel Winnik             TOR  Carey Price  ...           NaN   
4      Tip-In  Joffrey Lupul             TOR  Carey Price  ...           NaN   

  X_Coordinate  Y_Coordinate   Distance      Angle

## Question 4.1: Features management

In [43]:
df_train_val = df_train_val[["Game_ID", "Game_Time","Period_Number", "X_Coordinate", "Y_Coordinate", "Distance", "Angle", "Shot_or_Goal"]]
# "pwplay_RemainingTime", "pwplay_fplayer", "pwplay_eplayer"

## Question 4.2: Previous elements infos

In [44]:
df_train_val["Game_Time"] = (df_train_val["Game_Time"]*60).apply(lambda x: int(x))

In [69]:
df_train_val["Change_ID"] = [False]+[df_train_val["Game_ID"][i] == df_train_val["Game_ID"][i-1] for i in range(1, len(df_train_val))]

In [75]:
col = list()
for i in range(len(df_train_val.Shot_or_Goal)):
    if df_train_val.Change_ID[i]:
        col += [df_train_val.Shot_or_Goal[i-1]]
    else:
        col += [np.nan] 

df_train_val["Pre_type"] = col

In [76]:
col = list()
for i in range(len(df_train_val)):
    if df_train_val.Change_ID[i]:
        col += [df_train_val.X_Coordinate[i-1]]
    else:
        col += [np.nan] 

df_train_val["Pre_X"] = col

In [78]:
col = list()
for i in range(len(df_train_val)):
    if df_train_val.Change_ID[i]:
        col += [df_train_val.Y_Coordinate[i-1]]
    else:
        col += [np.nan] 

df_train_val["Pre_Y"] = col

In [86]:
df_train_val["Time_Since_LastEvnt"] = abs(df_train_val["Game_Time"].diff())
for i in range(len(df_train_val)):
    if df_train_val.Change_ID[i]:
        continue
    else:
        df_train_val.at[i, "Time_Since_LastEvnt"] = np.nan

In [85]:
col = list()
for i in range(len(df_train_val)):
    if df_train_val.Change_ID[i]:
        col += [abs(np.linalg.norm(np.array([df_train_val.X_Coordinate[i], df_train_val.Y_Coordinate[i]]) - np.array([df_train_val.Pre_X[i], df_train_val.Pre_Y[i]])))]
    else:
        col += [np.nan] 

df_train_val["Dist_from_LastEvnt"] = col

In [87]:
df_train_val.head()

,Game_ID,Game_Time,Period_Number,X_Coordinate,Y_Coordinate,Distance,Angle,Shot_or_Goal,Change_ID,Pre_type,Pre_X,Pre_Y,Dist_from_LastEvnt,Rebond,Changement_d'angle_de_tir,Time_Since_LastEvnt
0,2015020001,51,1,-55.0,6.0,35.510562,9.727579,Shot,False,NaN,NaN,NaN,NaN,False,0,NaN
1,2015020001,64,1,-79.0,0.0,11.000000,0.000000,Shot,True,Shot,-55.0,6.0,24.738634,True,1,13.0
2,2015020001,66,1,-75.0,-1.0,15.033296,-3.814075,Shot,True,Shot,-79.0,0.0,4.123106,True,1,2.0
3,2015020001,112,1,-55.0,-36.0,50.209561,-45.806929,Shot,True,Shot,-75.0,-1.0,40.311289,True,1,46.0
4,2015020001,156,1,-61.0,-1.0,29.017236,-1.974934,Shot,True,Shot,-55.0,-36.0,35.510562,True,1,44.0


## Question 4.3: New Features

In [82]:
df_train_val["Rebond"] = [df_train_val.Shot_or_Goal[i] == df_train_val.Pre_type[i] for i in range(len(df_train_val))]

In [83]:
df_train_val["Changement_d'angle_de_tir"] = [int(df_train_val.Rebond[i]) for i in range(len(df_train_val))]

In [88]:
df_train_val["Vitesse"] = df_train_val.Dist_from_LastEvnt/df_train_val.Time_Since_LastEvnt

In [91]:
df_train_val.head()

,Game_ID,Game_Time,Period_Number,X_Coordinate,Y_Coordinate,Distance,Angle,Shot_or_Goal,Change_ID,Pre_type,Pre_X,Pre_Y,Dist_from_LastEvnt,Rebond,Changement_d'angle_de_tir,Time_Since_LastEvnt,Vitesse
0,2015020001,51,1,-55.0,6.0,35.510562,9.727579,Shot,False,NaN,NaN,NaN,NaN,False,0,NaN,NaN
1,2015020001,64,1,-79.0,0.0,11.000000,0.000000,Shot,True,Shot,-55.0,6.0,24.738634,True,1,13.0,1.902972
2,2015020001,66,1,-75.0,-1.0,15.033296,-3.814075,Shot,True,Shot,-79.0,0.0,4.123106,True,1,2.0,2.061553
3,2015020001,112,1,-55.0,-36.0,50.209561,-45.806929,Shot,True,Shot,-75.0,-1.0,40.311289,True,1,46.0,0.876332
4,2015020001,156,1,-61.0,-1.0,29.017236,-1.974934,Shot,True,Shot,-55.0,-36.0,35.510562,True,1,44.0,0.807058


## Question 4.5: Comet 

In [90]:
from comet_ml import Experiment

experiment = Experiment(
    api_key="uHUS5l41WlDGkxYpJJMkBaLYd",
    project_name="ift-6758-a22",
    workspace="teylouniseifu",
)
experiment.log_dataframe_profile(
    df_train_val.loc[df_train_val.Game_ID == 2017021065],
    name='wpg_v_wsh_2017021065', # keep this name
    dataframe_format='csv' # ensure you set this flag!
)

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/teylouniseifu/ift-6758-a22/f80d9387fd13412fa8cd14de5f01f52a
COMET INFO:   Uploads:
COMET INFO:     dataframe                : 1 (9.97 KB)
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (717.58 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     notebook                 : 1
COMET INFO:     source_code              : 1
COMET INFO: ---------------------------
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/teylouniseif

{'profile': None,
 'dataframe': {'web': 'https://www.comet.com/api/asset/download?assetId=c10bd558e06a4bf799c7b8ae0aabb33f&experimentKey=2ff23b4950624386815d29f852b553c4',
  'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=c10bd558e06a4bf799c7b8ae0aabb33f&experimentKey=2ff23b4950624386815d29f852b553c4',
  'assetId': 'c10bd558e06a4bf799c7b8ae0aabb33f'}}